In [ ]:
# !pip install psycopg2-binary

In [91]:
import os
import re 

import psycopg2

In [2]:
db_params = {
    "dbname": "",
    "user": "",
    "password": "",
    "host": "",
    "port": ""
}

In [3]:
content_path = "./text/recipe/"

In [4]:
filenames = []
for i in range(1, 351):
    filenames.append(f"{i}.txt")

In [5]:
# recipe table
insert_query = "INSERT INTO recipe (id, name, path_photo, creater_id) VALUES (%s, %s, %s, %s)"

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

for filename in filenames:
        file_path = os.path.join(content_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

            recipe_name = file_content.split("📖")[0].replace("⠀", "").strip()

            recipe_id = int(filename.split('.')[0].strip())
            path_photo = filename.split('.')[0].strip() + ".jpg"

            cur.execute(insert_query, (recipe_id, recipe_name, path_photo, 2))

conn.commit()
cur.close()
conn.close()

In [94]:
# ingridient table
def extract_ingredients(text):
    ingredients_block = re.search(r"Ингредиенты:(.*?)🧑🏻‍🍳", text, re.DOTALL)
    if not ingredients_block:
        return []

    ingredients_text = ingredients_block.group(1)

    ingredients = re.findall(r"(?<=- )([А-ЯЁ][\w\s,.-]*?(?=\s- [А-ЯЁ]|$))", ingredients_text)

    ingredients = [ingredient.strip() for ingredient in ingredients]

    return ingredients

insert_query = "INSERT INTO ingredient (info, recipe_id) VALUES (%s, %s)"

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

for filename in filenames:
        file_path = os.path.join(content_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

            recipe_ingredients = extract_ingredients(file_content)

            recipe_id = int(filename.split('.')[0].strip())

            for ingridient in recipe_ingredients:
                cur.execute(insert_query, (ingridient, recipe_id))

conn.commit()
cur.close()
conn.close()

In [168]:
# step table
def extract_steps(text):
    match = re.search(r'Приготовление:s*(.+)', text, re.DOTALL)
    if match:
        preparation_text = match.group(1)

        steps = re.findall(r'(\d+\.s*[^.]+(?:.(?!s*d+.))?)+', preparation_text)

        steps = [re.sub(r'^d+.s*', '', step).strip() for step in steps]

        steps = [step[2:].strip() for step in steps]
        
        return steps
    return []

insert_query = "INSERT INTO step (info, recipe_id) VALUES (%s, %s)"

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

for filename in filenames:
        file_path = os.path.join(content_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

            recipe_steps = extract_steps(file_content)

            recipe_id = int(filename.split('.')[0].strip())

            for step in recipe_steps:
                cur.execute(insert_query, (step, recipe_id))

conn.commit()
cur.close()
conn.close()
